In [1]:
import os.path as op
import warnings

import numpy as np
import pandas as pd
from scipy.signal import correlate, hilbert
import scipy.ndimage as nd

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import get_scorer, make_scorer, r2_score, mean_absolute_error, mean_squared_log_error
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor

from utilities import reduce_df_mem_usage, print_score

from matplotlib import pyplot as plt
import seaborn as sns

import gc

In [2]:
#sns.set("dark_background")
#sns.set(context="talk")
gc.enable()

sns.set(style="ticks", context="talk")
plt.style.use("dark_background")
%matplotlib inline

rand_seed = 1234
rand_state = np.random.RandomState(rand_seed)

warnings.simplefilter(action='ignore', category=FutureWarning)

PATH = '/home/dhren/Documents/000_flatiron/002_projects/006_1028_kaggle_ASHRAE/data'

FIGSIZE = (28, 20)

In [17]:
def partition_idx(l, n):
    q,r = divmod(l, n) 
    return [i * q + min(i,r) for i in range(n+1)] 

def partition_lst(lst, n):
    idx = partition_idx(len(lst), n)
    return [lst[idx[i]: idx[i+1]] for i in range(n)]

In [22]:
%%time

df_tmp = pd.read_csv(op.join(PATH, 'train.csv'))

bids = df_tmp.building_id.sort_values().unique()

for i, batch in enumerate(partition_lst(bids, 10)):
    df_tmp[df_tmp.building_id.isin(batch)].to_csv(op.join(PATH, 'train_{:02d}.csv'.format(i)), index = False)

In [23]:
%%time

df_tmp = pd.read_csv(op.join(PATH, 'test.csv'))

for i, batch in enumerate(partition_lst(bids, 10)):
    df_tmp[df_tmp.building_id.isin(batch)].to_csv(op.join(PATH, 'test_{:02d}.csv'.format(i)), index = False)

CPU times: user 2min 42s, sys: 3.11 s, total: 2min 45s
Wall time: 2min 38s
